# Story2CQ LLM Generation

In [1]:
from google import genai
from google.genai import types

from utils import get_key, generate_hash

In [2]:
PERSONA_FILES = {
    "sonia" :"../data/bme_persona_sonia.md",
    "liz": "../data/bme_persona_liz.md",
}
SELECTED_USER_STORY = "../data/bme_us1.md"

In [3]:
persona_descriptions = {}

for persona_name, persona_file in PERSONA_FILES.items():
    with open(persona_file, "r") as f:
        persona_descriptions[persona_name] = f.read()
        
# Read the selected user story
with open(SELECTED_USER_STORY, "r") as f:
    user_story = f.read()

## Prompt creation

In [4]:
SYSTEM_ROLE = "You are an ontology engineer who is tasked to formulate requirements in the form of competency question given a set of persona descriptions and user stories."

INSTRUCTION = "Generate competency questions based on the provided persona descriptions and user story."

# Combine the persona descriptions and user story into a single input
prompt = INSTRUCTION + "\n\n[Persona Descriptions]\n"
for _, persona_description in persona_descriptions.items():
    prompt += f"{persona_description}\n\n"
prompt += "[User Story]\n" + user_story

## Gemini extraction

In [7]:
LLM_CONFIG = {
    "temperature": 0,
    "max_tokens": 2000,
    "top_p": 1,
    "frequency_penalty": 0,
    "presence_penalty": 0,
    "seed": 46, # the Dr.
}

GEMINI_MODEL = "gemini-1.5-pro"
GEMINI_KEY = get_key("gemini")

model_hash = generate_hash(LLM_CONFIG)

In [8]:
client = genai.Client(api_key=GEMINI_KEY)

In [9]:
response = client.models.generate_content(
    model=GEMINI_MODEL,
    config=types.GenerateContentConfig(
        system_instruction=SYSTEM_ROLE,
        temperature=LLM_CONFIG["temperature"],
        max_output_tokens=LLM_CONFIG["max_tokens"],
        top_p=LLM_CONFIG["top_p"],
        frequency_penalty=LLM_CONFIG["frequency_penalty"],
        presence_penalty=LLM_CONFIG["presence_penalty"],
        seed=LLM_CONFIG["seed"],
    ),
    contents=prompt,
)

In [11]:
# print(response.text)
# Dump the response to a file
with open(f"../data/bme_cqs_us1_{GEMINI_MODEL}_{model_hash}.txt", "w") as f:
    f.write(response.text)